In [ ]:
%run quantum_one_time_pad.ipynb
%run che_initialization.ipynb
%run epr_encryption.ipynb
%run epr_evaluation.ipynb

In [ ]:
def cl_dec(rho_hom_eval, a_tilde_update, b_tilde_update):
    """
    Perform decryption of the encrypted evaluated cipherstate according to the Clifford scheme and
    return the decrypted evaluated state.
    For more information refer to the Clifford scheme by Broadbent and Jeffery: https://arxiv.org/abs/1412.8766
    
    Args: 
        - rho_hom_eval: density matrix of the encrypted evaluated quantum state
        - a_tilde_update: a list of updated classical ciphertexts 
        which contain the information about the X-Pauli gate exponents 
        - b_tilde_update: a list of updated classical ciphertexts 
        which contain the information about the Z-Pauli gate exponents 
        
    Returns: 
        - rho_dec_eval: evaluated decrypted density matrix of the input message
    """
    # decrypt the ciphertexts
    a_decrypted = [decryptor.decrypt(i) for i in a_tilde_update]
    b_decrypted = [decryptor.decrypt(i) for i in b_tilde_update]
    
    a_decoded = [encoder.decode(i) for i in a_decrypted]
    b_decoded = [encoder.decode(i) for i in b_decrypted]
    
    # extract the first bit from each list
    a_dec = [i[0] for i in a_decoded]
    b_dec = [i[0] for i in b_decoded]
    
    rho_dec_eval = qotp_dec_density(rho_hom_eval, a_dec, b_dec)
    return rho_dec_eval

In [ ]:
def number_of_tgates_dictionary(circuit_dictionary):
    """
    Count the total number of T gates in a circuit from the circuit dictionary.
    
    Args: 
        - circuit_dictionary: a dictonary object where keys indicate the ordering of qubits, 
        starting from 1 as a string. The values associated with each key is a list which contains 
        the following information, in the same order:
            1) acting wire 
            2) a Boolean True/False value indicating whether a T gate has been applied during evaluation or not. 
            This value gets updated during the evaluation process.
            3) the total number of T gates that are going to be applied on the qubit as an integer. 
            4) the number of T gates applied at the end of the execution of each layer. This value gets updated
            during the evaluation process. 
        
    Returns:
        - number_of_t_gates: the total number of T gates in a circuit as an integer
    """
    number_of_t_gates = 0
    for key in circuit_dictionary:
        number_of_t_gates += circuit_dictionary[key][2]  # Access the third element of the list
    return number_of_t_gates

In [ ]:
def epr_dec(rho_hom_eval, a_tilde_update, b_tilde_update, circuit_dictionary, key_polynomials=None,t_gate_sequence_over_time=None, c_encrypted=None, p_exponents=None):
    """
    Perform decryption of the encrypted evaluated cipherstate according to the EPR scheme.
    If the evaluated circuit does not consist of any T gates, then an application of the 
    quantum one-time pad is sufficient for decryption. 
    If there is at least one T gate in the circuit, then phase gates, conditional 
    on whether a X Paulis were applied in the encryption of the quantum message, is required.
    A symbolic computation is required to assess whether a correction is needed for the application of 
    the phase gate. After running a circuit consisting of a potential Phase gate and a Hadamard gate, 
    a measurement is performed to obtain the unknown `k` value, which then will be stored in a list.
    At the end, one last check is required before the performing the quantum one-time pad on  
    `a_tilde_update` and `b_tilde_update`. The evaluated decrypted density matrix of the input message is returned
    at the end.
    For more information, refer to the EPR scheme by Broadbent and Jeffery: https://arxiv.org/abs/1412.8766. 
    
    
    Args: 
        - rho_hom_eval: density matrix of the encrypted evaluated quantum state
        - a_tilde_update: a list of updated classical ciphertexts 
        which contain the information about the X-Pauli gate exponents 
        - b_tilde_update: a list of updated classical ciphertexts 
        which contain the information about the Z-Pauli gate exponents 
        - circuit_dictionary: a dictonary object where keys indicate the ordering of qubits, starting from 1 as a string. 
        The values associated with each key is a list which contains the following information, in the same order:
            1) acting wire 
            2) a Boolean True/False value indicating whether a T gate has been applied during evaluation or not. 
            This value gets updated during the evaluation process.
            3) the total number of T gates that are going to be applied on the qubit as an integer. 
            4) the number of T gates applied at the end of the execution of each layer. This value gets updated
            during the evaluation process. 

        If there are any T gates in the circuit:
            - key_polynomials: a list containing the symbolic polynomial before the applicaton 
        of a T gate and all the key updates at the end of the circuit evaluation. 
            - t_gate_sequence_over_time: a nested list where each element is a list with the order of the qubit and the 
        order of the layer in which it appears in the circuit. 
            - c_encrypted: a list of ciphertexts resulting from the measurement during the evaluation of the EPR scheme
            - p_exponents: a list of ciphertexts recording the updated "a_tilde" value before the application of a
            T gate.
            
    Returns: 
        - rho_dec_eval: evaluated decrypted density matrix of the input message 
    """
    # making a copy of the `circuit_dictionary`
    acting_wire_dictionary = circuit_dictionary
    
    t_gate_count = number_of_tgates_dictionary(acting_wire_dictionary)
    if t_gate_count == 0:
        return cl_dec(rho_hom_eval, a_tilde_update, b_tilde_update)
    
    # Define variables for bookkeeping
    t_index_list = tgate_index(acting_wire_dictionary)
    number_of_qubits = int(list(acting_wire_dictionary.keys())[-1])
    number_of_wires = int(np.log2(rho_hom_eval.shape[0]))
    c_dec = [[] for i in range(number_of_qubits)]
    k_measurements = [[] for i in range(number_of_qubits)]
    phase_dec = [[] for i in range(number_of_qubits)]
    
    
    # decrypt the ciphertexts
    a_dec = [encoder.decode(decryptor.decrypt(i))[0]%2 for i in a_tilde_update]
    b_dec = [encoder.decode(decryptor.decrypt(i))[0]%2 for i in b_tilde_update]
    for qubit_order in range(len(t_index_list)):
        c_dec[t_index_list[qubit_order]] = [encoder.decode(decryptor.decrypt(j))[0] for j in c_encrypted[t_index_list[qubit_order]]]
        phase_dec[t_index_list[qubit_order]] = [encoder.decode(decryptor.decrypt(j))[0] % 2 for j in p_exponents[t_index_list[qubit_order]]]

    rho_dec_eval = rho_hom_eval
    
    
    # decryption begins here
    for i in range(len(t_gate_sequence_over_time)):
        qubit_order = t_gate_sequence_over_time[i][0]

        if acting_wire_dictionary[str(qubit_order+1)][3] == 0:
            acting_wire_dictionary[str(qubit_order+1)][1] = True 
            acting_wire_dictionary[str(qubit_order+1)][3] = acting_wire_dictionary[str(qubit_order+1)][3] + 1
            no_t_gates_before_ith_qubit = number_of_t_gates_applied(acting_wire_dictionary, qubit_order)
            acting_wire = qubit_order + 2*no_t_gates_before_ith_qubit + 3
            temp_list = acting_wire_dictionary[str(qubit_order+1)]
            temp_list.append(acting_wire)
            acting_wire_dictionary[str(qubit_order+1)] = temp_list

        else:
            acting_wire = acting_wire_dictionary[str(qubit_order+1)][4] + 2
            acting_wire_dictionary[str(qubit_order+1)][3] = acting_wire_dictionary[str(qubit_order+1)][3] + 1
            acting_wire_dictionary[str(qubit_order+1)][4] = acting_wire
            
        t_gate_count_on_ith_qubit = acting_wire_dictionary[str(qubit_order+1)][3] - 1
        current_polynomial = key_polynomials[qubit_order][t_gate_count_on_ith_qubit]

        phase_gate_exponent_correction = evaluate_expression(current_polynomial, k_measurements, c_dec)
        p_exponent_value = phase_dec[qubit_order][t_gate_count_on_ith_qubit] ^ phase_gate_exponent_correction

        p_operator = np.kron(iterated_tensor_prod(I,acting_wire-1), 
                             matrix_power(P, p_exponent_value))
        p_operator = np.kron(p_operator, iterated_tensor_prod(I,number_of_wires-acting_wire))
        phase_on_rho_dec_eval = qgate_on_density(p_operator, rho_dec_eval)


        h_operator = np.kron(iterated_tensor_prod(I,acting_wire-1), H)
        h_operator = np.kron(h_operator, iterated_tensor_prod(I,number_of_wires-acting_wire))
        h_on_phase_rho_dec_eval = qgate_on_density(h_operator, phase_on_rho_dec_eval)


        pms, k = post_measurement_state(h_on_phase_rho_dec_eval, 
                                       [acting_wire - 1],  # -1 to account for acting_wire starts from 1
                                       # python indexing starts from 0
                                       measurement(partial_trace(h_on_phase_rho_dec_eval, [acting_wire - 1])))


        k_measurements[qubit_order].append(int(k))
        rho_dec_eval = pms

    # applying necessary correction to the last layer of the circuit
    for qubit_order in range(number_of_qubits):
        a_exponent_poly = key_polynomials[qubit_order][-1][0]
        b_exponent_poly = key_polynomials[qubit_order][-1][1]
        a_exponent_correction = evaluate_expression(a_exponent_poly, k_measurements, c_dec)
        b_exponent_correction = evaluate_expression(b_exponent_poly, k_measurements, c_dec)

        a_dec[qubit_order] = a_dec[qubit_order] ^ a_exponent_correction
        b_dec[qubit_order] = b_dec[qubit_order] ^ b_exponent_correction

    # extracting the order of wires that are not traced out after partial trace
    acting_wires = []        
    for i in range(number_of_qubits):
        if len(acting_wire_dictionary[str(i+1)]) == 5:
            acting_wires.append(acting_wire_dictionary[str(i+1)][4]-2)
        if len(acting_wire_dictionary[str(i+1)]) == 4:
            acting_wires.append(i + 2 * number_of_t_gates_applied(acting_wire_dictionary, i))
            
    rho_dec_eval = qotp_dec_density(partial_trace(rho_dec_eval, acting_wires), 
                                     a_dec, 
                                     b_dec)
    
    return rho_dec_eval